In [2]:
import os
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool
from crewai.tools import BaseTool
from langchain_community.utilities.dalle_image_generator import DallEAPIWrapper
import dotenv
from datetime import datetime
import requests
import base64
from typing import Optional, ClassVar, List
from pydantic import Field, PrivateAttr
from openai import OpenAI
from pathlib import Path
from jinja2 import Template
from IPython.display import HTML, display
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import tempfile
import time

from utils.linkedin_preview import LinkedInPreviewGenerator
from tools.image_generation.image_gen import ImageGenerationTool


from constants.content_requirements import (
    Platform,
    ExpertiseLevel,
    PLATFORM_FORMATS,
    EXPERTISE_REQUIREMENTS
)
from constants.domains import PREFERRED_DOMAINS

In [11]:
image_designer_agent = Agent(
    role='Image Designer',
    goal='Create subtle, text-friendly background images for social media content',
    backstory=(
        "An expert in creating minimalist, professional background designs "
        "that enhance readability and maintain visual hierarchy. Specializes in "
        "subtle patterns, gradients, and abstract compositions that complement "
        "text overlays without competing for attention."
    ),
    verbose=True,
    memory=True,
    tools=[ImageGenerationTool()]
)

image_design_task = Task(
    description=(
        "Create background images for a {social_platform} carousel post about {topic} "
        "targeting {expertise_level} audience. Generate one image per slide."
        "\n\nImage requirements:"
        "\n- Extremely subtle and minimal background designs"
        "\n- NO human figures, hands, or detailed objects"
        "\n- Focus on abstract patterns, gentle gradients, or simple geometric shapes"
        "\n- Ensure high text readability with clean, uncluttered compositions"
        "\n- Use muted colors that won't compete with text overlays"
        "\n- Color scheme based on complexity level:"
        "\n  * Beginner: Soft, warm gradients (blues, warm grays)"
        "\n  * Intermediate: Professional, neutral tones (navy, slate, subtle gold)"
        "\n  * Advanced: Rich, deep colors (dark blues, burgundy, charcoal)"
        "\n\nAdditional guidelines:"
        "\n- Maintain 30% or less visual complexity"
        "\n- Ensure patterns are subtle enough to read white or black text clearly"
        "\n- Avoid any text or symbols in the images"
    ),
    expected_output="A list of paths to generated images, one for each carousel slide.",
    agent=image_designer_agent
)

In [12]:

research_agent = Agent(
    role='Researcher',
    goal='Gather detailed and relevant information about {topic}.',
    backstory=(
        "A skilled and inquisitive researcher, adept at diving deep into topics "
        "and pulling out critical insights that are accurate and comprehensive. "
        "Prioritizes authoritative sources and cross-references information."
    ),
    verbose=True,
    memory=True,
    tools=[
        SerperDevTool(
            search_params={
                "site": " OR ".join(PREFERRED_DOMAINS.get("trading", [])),
                "num": 5
            }
        ),
        SerperDevTool(
            search_params={
                "num": 5,
                "exclude": "pinterest.com,facebook.com,instagram.com"
            }
        )
    ]
)

writer_agent = Agent(
    role='Writer',
    goal='Create content about {topic} tailored for {expertise_level} audience on {social_platform}.',
    backstory=(
        "An expert content creator who specializes in adapting complex topics for different "
        "audiences and social media platforms. Skilled at creating engaging content that "
        "matches each platform's unique style and requirements."
    ),
    verbose=True,
    memory=True,
)


research_task = Task(
    description=(
        "Conduct thorough research on the topic {topic}. Focus on collecting key points, "
        "relevant statistics, and critical insights to form a strong foundation for the report."
    ),
    expected_output="A comprehensive list of data points and insights about {topic}.",
    agent=research_agent
)
def create_writing_task(platform: Platform, expertise_level: ExpertiseLevel) -> Task:
    """
    Creates a writing task with specific requirements based on platform and expertise level
    """
    platform_format = PLATFORM_FORMATS.get(platform.value, {}).get("description", "")
    expertise_reqs = EXPERTISE_REQUIREMENTS.get(expertise_level.value, {})
    
    task_description = (
        "Using the research findings, create content about {topic} for {social_platform}. "
        f"The content should be appropriate for a {expertise_level} audience with a "
        f"{expertise_reqs.get('tone', 'neutral')} tone.\n\n"
        f"{platform_format}\n\n"
        f"{expertise_reqs.get('description', '')}\n\n"
        "Ensure the content is engaging, accurate, and matches the platform's style."
    )
    
    return Task(
        description=task_description,
        expected_output=f"Platform-native content formatted for {platform}",
        agent=writer_agent,
        async_execution=False
    )

writing_task = create_writing_task(
    platform=Platform.LINKEDIN,
    expertise_level=ExpertiseLevel.ADVANCED
)


content_crew = Crew(
    agents=[research_agent, writer_agent],
    tasks=[research_task, writing_task],
    process=Process.sequential
)

2024-12-18 14:25:43,329 - 8117181504 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


In [13]:
content_crew = Crew(
    agents=[research_agent, writer_agent],
    tasks=[research_task, writing_task],
    process=Process.sequential
)

# Generate the content first
content_result = content_crew.kickoff(inputs={
    "topic": "trading crude oil futures",
    "expertise_level": "advanced",
    "social_platform": "linkedin"
})



2024-12-18 14:25:48,758 - 8117181504 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Researcher
## Task: Conduct thorough research on the topic trading crude oil futures. Focus on collecting key points, relevant statistics, and critical insights to form a strong foundation for the report.


# Agent: Researcher
## Thought: I need to gather detailed information about trading crude oil futures, including key points, relevant statistics, and insights that will support a comprehensive understanding of the topic.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"trading crude oil futures key points statistics insights\"}"
## Tool Output: 

Search results: Title: Crude Oil Futures Overview - CME Group
Link: https://www.cmegroup.com/markets/energy/crude-oil/light-sweet-crude.html
Snippet: Deep, liquid market. Over 1 million contracts of WTI futures and options trade daily, with approximately 4 million contracts of open interest.
---
Title: API Data Reveals Sharp Draw in U.S. Crude Stocks, Boosting Futures
Link: https://www.fxempire.com/forecasts

In [5]:

image_crew = Crew(
    agents=[image_designer_agent],
    tasks=[image_design_task],
    process=Process.sequential
)

# Generate images based on the content
image_result = image_crew.kickoff(inputs={
    "topic": "trading S&P 500 futures",
    "expertise_level": "advanced",
    "social_platform": "linkedin",
    "content": content_result
})

print("Content:", content_result)
print("Image paths:", image_result)

2024-12-18 13:13:35,725 - 8117181504 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Image Designer
## Task: Create background images for a linkedin carousel post about trading S&P 500 futures targeting advanced audience. Generate one image per slide.

Image requirements:
- Extremely subtle and minimal background designs
- NO human figures, hands, or detailed objects
- Focus on abstract patterns, gentle gradients, or simple geometric shapes
- Ensure high text readability with clean, uncluttered compositions
- Use muted colors that won't compete with text overlays
- Color scheme based on complexity level:
  * Beginner: Soft, warm gradients (blues, warm grays)
  * Intermediate: Professional, neutral tones (navy, slate, subtle gold)
  * Advanced: Rich, deep colors (dark blues, burgundy, charcoal)

Additional guidelines:
- Maintain 30% or less visual complexity
- Ensure patterns are subtle enough to read white or black text clearly
- Avoid any text or symbols in the images
 

I encountered an error while trying to use the tool. This was the error: Arguments valida

In [14]:
from utils.linkedin_preview import LinkedInPreviewGenerator

linkedin_preview_generator = LinkedInPreviewGenerator()

linkedin_preview_generator.generate_preview(content_result)